In [33]:
import sys
sys.path.append('../')
from EnsembleLearning.decision_tree import DecisionTree
from EnsembleLearning.AdaBoost import AdaBoostTree
from EnsembleLearning.BaggedTrees import BaggedTrees, RandomForest

In [34]:
import pandas as pd
import numpy as np
import csv

In [51]:
df_train = pd.read_csv('train_final.csv')
df_train = df_train.rename(columns={'fnlwgt': 'weights', 'income>50K': 'label'})
# df_train['weights']=df_train['weights'] / df_train['weights'].sum()
df_train['label'] = df_train['label'].apply(lambda x : -1 if x == 0 else 1)
df_test = pd.read_csv('test_final.csv')
df_test = df_test.rename(columns={'fnlwgt': 'weights'})
# df_test['weights']=df_test['weights'] / df_test['weights'].sum()


In [27]:
def encode_age(x):
    if x < 28:
        return 'under_28'
    elif x >= 28 and x < 37:
        return '28-37'
    elif x >= 37 and x < 48:
        return '37-48'
    elif x >= 48:
        return 'over_48'

In [28]:
def encode_hours(x):
    if x < 40:
        return 'under_40'
    elif x >= 28 and x < 45:
        return '40-45'
    elif x >= 45:
        return 'over_45'

In [7]:
df_train['capital.loss'] = df_train['capital.loss'].apply(lambda x: 'loss' if x > 0 else 'no_loss')
df_test['capital.loss'] = df_test['capital.loss'].apply(lambda x: 'loss' if x > 0 else 'no_loss')

df_train['capital.gain'] = df_train['capital.gain'].apply(lambda x: 'gain' if x > 0 else 'no_gain')
df_test['capital.gain'] = df_test['capital.gain'].apply(lambda x: 'gain' if x > 0 else 'no_gain')

df_train['age'] = df_train['age'].apply(encode_age)
df_test['age'] = df_test['age'].apply(encode_age)

df_train['hours.per.week'] = df_train['hours.per.week'].apply(encode_hours)
df_test['hours.per.week'] = df_test['hours.per.week'].apply(encode_hours)


df_train = df_train.drop('education', axis=1)
df_test = df_test.drop('education', axis=1)


In [8]:
attributes = df_train.columns.tolist()
attributes.remove('weights')
attributes.remove('label')

In [9]:
df_train

,age,workclass,weights,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,label
0,over_48,Self-emp-not-inc,0.000020,15,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,no_gain,no_loss,40-45,India,1
1,28-37,Self-emp-not-inc,0.000026,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,no_gain,no_loss,40-45,United-States,1
2,37-48,Private,0.000031,9,Married-civ-spouse,Craft-repair,Husband,White,Male,no_gain,no_loss,40-45,United-States,-1
3,37-48,Private,0.000024,9,Divorced,Craft-repair,Other-relative,White,Female,no_gain,no_loss,40-45,Vietnam,-1
4,37-48,Private,0.000024,9,Married-civ-spouse,Transport-moving,Husband,White,Male,no_gain,no_loss,over_45,United-States,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,under_28,Private,0.000018,9,Never-married,Adm-clerical,Not-in-family,White,Female,no_gain,no_loss,under_40,United-States,-1
24996,over_48,Local-gov,0.000043,9,Married-civ-spouse,Adm-clerical,Wife,White,Female,no_gain,no_loss,under_40,United-States,-1
24997,37-48,Private,0.000048,14,Married-civ-spouse,Transport-moving,Husband,White,Male,no_gain,no_loss,40-45,Poland,-1
24998,over_48,Private,0.000073,9,Divorced,Prof-specialty,Not-in-family,White,Male,gain,no_loss,over_45,United-States,1


In [123]:
dtree = DecisionTree(df_train, attributes, weighted=True).build_tree(purity_type='entropy', max_depth=20)

In [124]:
df_test['preds'] = df_test.apply(lambda row : dtree.predict(row.drop('ID')), axis=1)
preds = df_test['preds'].tolist()


In [10]:
bagged_trees = RandomForest(df_train, attributes)
bagged_trees.build_trees(100, 5)

In [11]:
df_test['native.country'] = df_test['native.country'].apply(lambda x : x if x != 'Holand-Netherlands' else 'Germany')

In [12]:
df_test['preds'] = df_test.apply(lambda row : bagged_trees.predict(row.drop('ID')), axis=1)
df_test['preds'] = df_test['preds'].apply(lambda x : 0 if x == -1 else 1)
preds = df_test['preds'].tolist()

In [13]:
import csv
ctr = 1
data = [['ID', 'Prediction']]
for pred in preds:
    row = [ctr, pred]
    data.append(row)
    ctr += 1
# open the file in the write mode
with open('results_randomforest.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)